<a href="https://colab.research.google.com/github/DeependraChaddha/RAG_Projects/blob/main/RAG_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##ENVIRONMENT

INSTALLING PACKAGES

In [ ]:
!pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain youtube-transcript-api pytube

SETTING UP LANGSMITH

In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2']='true'
os.environ['LANGCHAIN_ENDPOINT']= 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY']=###YOUR API KEY###

SETTING OPENAI

In [ ]:
os.environ['OPENAI_API_KEY']=###YOUR API KEY###

##MULTI-REPRESENTATION INDEXES

LOAD REQUIRED DOCUMENTS

In [ ]:
#make imports
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

#load the blog
loader= WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs=loader.load()

loader=WebBaseLoader("https://lilianweng.github.io/posts/2024-02-05-human-data-quality/")
docs.extend(loader.load())

MAKE RAG CHAIN

In [ ]:
#make imports
import uuid
from langchain.documents import documents
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

#make RAG chain
chain=(
    {"doc":lambda x: x.page_content}
    | ChatPromptTemplate.from_template("Summarize the following document:\n\n{doc}")
    |ChatOpenAI(model="gpt-3.5-turbo", max_retries=0)
    |StrOutputParser()
)

summaries=chain.batch(docs,{"max_concurrency":5})

EMBED SUMMARY DOCUMENTS

In [ ]:
#make imports
from langchain.imports import InMemoryByteStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.retrievers.multi_vector import MultiVectorRetriever

#vectorstore to store the chunks
vectorstore=Chroma(collection_name="summaries",
                   embedding_function=OpanAIEmbeddings())

#storing parent documents
store=InMemoryByteStore()
id_key="doc_id"

#define retriever
retriever=MultiVectorRetriever(vectorstore=vectorstore,
                               byte_store=store,
                               id_key=id_key,
                               )

doc_ids=[str(uuid.uuid4()) for _ in docs]

#make list of summary docs
summary_docs=[Document(page_content=s,metadata={id_key:doc_ids[i]}) for i,s in enumerate(summaries)]

#Retriever for documents
retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_ids,docs)))

In [ ]:
query="Memory in Agents"
sub_docs=vectorstore.similarity_search(query,k=1)
sub_docs[0]

In [ ]:
retrieved_docs=retriever.get_relevant_documents(query,n_results=1)
retrieved_docs[0].page_content[0:500]